# Creating a Reusable and Modular DAO Layer in Python using SQLAlchemy

This setup segregates configurations, database initialization, base models, session management, and the Data Access Object (DAO) layer to achieve a clean, modular, and reusable design.

## 1. Configurations and Settings

Here's how you could approach it:

### Steps:

1. **Configurations and Settings**
2. **Database Initialization**
3. **Base Model Definition**
4. **Session Management**
5. **DAO Layer**
6. **Full Implementation**

In [ ]:
# settings.py

class Settings:
    DEFAULT_SQLALCHEMY_DATABASE_URI = 'sqlite:///./test.db'

settings = Settings()

2. **Database Initialization**

In [ ]:
# db.py

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from contextlib import contextmanager
from setting import settings

SQLALCHEMY_DATABASE_URL = settings.DEFAULT_SQLALCHEMY_DATABASE_URI
engine = create_engine(SQLALCHEMY_DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

@contextmanager
def session_scope():
    session = SessionLocal()
    try:
        yield session
        session.commit()
    except Exception as e:
        session.rollback()
        raise
    finally:
        session.close()


3. **Base Model Definition**

In [ ]:
# base.py

from sqlalchemy.ext.declarative import as_declarative, declared_attr

@as_declarative()
class Base:
    id: Any
    __name__: str

    @declared_attr
    def __tablename__(cls) -> str:
        return cls.__name__.lower()


## 4. Session Management (Revised)

Refer to `db.py` above for the `session_scope`.

## 5. Implementing the DAO Layer

### Generic DAO Implementation:

In [ ]:
# dao.py

from typing import Type, TypeVar, Generic, List
from sqlalchemy.orm import Session
from base import Base

T = TypeVar('T', bound=Base)

class GenericDAO(Generic[T]):
    def __init__(self, model: Type[T]):
        self._model = model

    def get_all(self, db: Session) -> List[T]:
        return db.query(self._model).all()

    def get(self, db: Session, entity_id: Any) -> T:
        return db.query(self._model).filter(self._model.id == entity_id).first()

    def create(self, db: Session, obj_in: T) -> T:
        db.add(obj_in)
        db.commit()
        db.refresh(obj_in)
        return obj_in

    def update(self, db: Session, db_obj: T, obj_in) -> T:
        for key, value in obj_in.items():
            setattr(db_obj, key, value)
        db.commit()
        db.refresh(db_obj)
        return db_obj

    def remove(self, db: Session, entity_id: Any) -> T:
        obj = db.query(self._model).get(entity_id)
        db.delete(obj)
        db.commit()
        return obj


## Example Usage

### Define Your Models

In [ ]:
# models.py

from sqlalchemy import Column, Integer, String
from base import Base

class User(Base):
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)
    email = Column(String, unique=True, index=True)


### Using the DAO

In [ ]:
# main.py

from db import session_scope
from models import User
from dao import GenericDAO

def main():
    user_dao = GenericDAO(User)
    
    with session_scope() as session:
        new_user = User(name="John Doe", email="john.doe@example.com")
        user_dao.create(session, new_user)

        all_users = user_dao.get_all(session)
        print(all_users)

if __name__ == "__main__":
    main()


## Conclusion

By segregating the code into configuration, database initialization, base models, session management, and the DAO layer, we achieve a clean, modular, and reusable design, making it easier to manage and extend.